## 1. Importar las Librerías Necesarias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier

%matplotlib inline
pd.set_option('display.max_columns', 30)
plt.rcParams['figure.figsize'] = [12.0, 8.0]

## 2. Leer los Datos
Al menos los datos Tabulares de la base de "train"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Labo2/2024/tp_final.ar/Data/train/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Labo2/2024/tp_final.ar/Data/test/test.csv')
train['dataset_type'] = 'train'
test['dataset_type'] = 'test'
all_data = pd.concat([train, test]) # Dataset completo

In [ ]:
#por el momento solo leemos los datos tabulares
# train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')

## 3. Pre-procesar Nulos
Verificar la existencia de Nulos y decidir como Imputarlos en caso de que existan

Verificar la existencia de Ceros u otros valores que puedan indicar que pueden ser perdidos

In [4]:
#vemos primero que valiables tienen nulos
train.isnull().sum()

#al parecer solo nombre y descripcion tienen nulos. Es por eso que vamos a considerar no tener nombre y no tener descripcion
#como una variable mas para usar de input del modelo.

,0
Type,0
Name,1265
Age,0
Breed1,0
Breed2,0
Gender,0
Color1,0
Color2,0
Color3,0
MaturitySize,0


In [5]:
train['Tiene_nombre?'] = np.where(train['Name'].notnull(), 0, 1)
train['Tiene_descripcion?'] = np.where(train['Description'].notnull(), 0, 1)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Type                14993 non-null  int64  
 1   Name                13728 non-null  object 
 2   Age                 14993 non-null  int64  
 3   Breed1              14993 non-null  int64  
 4   Breed2              14993 non-null  int64  
 5   Gender              14993 non-null  int64  
 6   Color1              14993 non-null  int64  
 7   Color2              14993 non-null  int64  
 8   Color3              14993 non-null  int64  
 9   MaturitySize        14993 non-null  int64  
 10  FurLength           14993 non-null  int64  
 11  Vaccinated          14993 non-null  int64  
 12  Dewormed            14993 non-null  int64  
 13  Sterilized          14993 non-null  int64  
 14  Health              14993 non-null  int64  
 15  Quantity            14993 non-null  int64  
 16  Fee 

In [7]:
train.describe()

###outliers###

#Age
#no vamos a hacer ningun tratamiento de esto, dado que para lightgbm no es necesario

###varialbes a Codificar###

#maturitysyze
#Vaccinated
#Dewormed
#sterilized
#health

####variable nueva###

#vacunated & dewormed
#vacunated & sterilezed
#sterilezed & dewormed
#sterlezed & dewormed & vacunated
#has photo?
#Has video?
#Has photo & video_?


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed,Tiene_nombre?,Tiene_descripcion?
count,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000
mean,1.457614,10.452078,265.272594,74.009738,1.776162,2.234176,3.222837,1.882012,1.862002,1.467485,1.731208,1.558727,1.914227,1.036617,1.576069,21.259988,41346.028347,0.056760,3.889215,2.516441,0.084373,0.000867
std,0.498217,18.155790,60.056818,123.011575,0.681592,1.745225,2.742562,2.984086,0.547959,0.599070,0.667649,0.695817,0.566172,0.199535,1.472477,78.414548,32.444153,0.346185,3.487810,1.177265,0.277955,0.029434
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41324.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,2.000000,2.000000,0.000000,0.000000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,3.000000,2.000000,0.000000,0.000000
75%,2.000000,12.000000,307.000000,179.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,41401.000000,0.000000,5.000000,4.000000,0.000000,0.000000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,20.000000,3000.000000,41415.000000,8.000000,30.000000,4.000000,1.000000,1.000000


In [8]:
###varialbes a Codificar###
OneHotEncoder().fit_transform(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])

#codificacion de variables

encoder = OneHotEncoder().fit(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]])


In [9]:
codificadas = pd.DataFrame(encoder.transform(train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]]).toarray(),
            columns=[
                f"{column}_{category}" for categories, column in zip(encoder.categories_, train[["MaturitySize", "Vaccinated","Dewormed","Sterilized", "Health"]].columns)
                for category in categories
            ])

In [10]:
train.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,dataset_type,Tiene_nombre?,Tiene_descripcion?
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,train,0,0
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,train,0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,train,0,0
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,train,0,0
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,train,0,0


In [11]:
train = pd.concat([train, codificadas], axis=1)

In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14993 entries, 0 to 14992
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Type                14993 non-null  int64  
 1   Name                13728 non-null  object 
 2   Age                 14993 non-null  int64  
 3   Breed1              14993 non-null  int64  
 4   Breed2              14993 non-null  int64  
 5   Gender              14993 non-null  int64  
 6   Color1              14993 non-null  int64  
 7   Color2              14993 non-null  int64  
 8   Color3              14993 non-null  int64  
 9   MaturitySize        14993 non-null  int64  
 10  FurLength           14993 non-null  int64  
 11  Vaccinated          14993 non-null  int64  
 12  Dewormed            14993 non-null  int64  
 13  Sterilized          14993 non-null  int64  
 14  Health              14993 non-null  int64  
 15  Quantity            14993 non-null  int64  
 16  Fee 

In [13]:
####variable nueva###
train['Vaccinated&Dewormed'] = np.where((train['Vaccinated'] == 1) & (train['Dewormed']==1), 1, 0)
train['Vaccinated&Sterilized'] = np.where((train['Vaccinated'] == 1) & (train['Sterilized']==1), 1, 0)
train['Sterilized&Dewormed'] = np.where((train['Sterilized'] == 1) & (train['Dewormed']==1), 1, 0)
train['Sterilized&Dewormed&Vaccinated'] = np.where((train['Vaccinated'] == 1) & (train['Dewormed']==1)& (train['Sterilized']==1), 1, 0)
train['Hasphoto?'] = np.where((train['PhotoAmt'] > 0), 1, 0)
train['Hasvideo?'] = np.where((train['VideoAmt'] > 0), 1, 0)
train['Hasvideo&photo?'] = np.where((train['VideoAmt'] > 0) & (train['PhotoAmt'] > 0), 1, 0)


## 4. Convertir o eliminar las Columnas Categóricas

Por ejemplo, la Descripción habría que sacarla para un análisis independiente

In [14]:
#variables a eliminar
eliminar = ["Name","RescuerID","Description","PetID"]
data = train.drop(eliminar, axis=1)
print(train.columns.values)

['Type' 'Name' 'Age' 'Breed1' 'Breed2' 'Gender' 'Color1' 'Color2' 'Color3'
 'MaturitySize' 'FurLength' 'Vaccinated' 'Dewormed' 'Sterilized' 'Health'
 'Quantity' 'Fee' 'State' 'RescuerID' 'VideoAmt' 'Description' 'PetID'
 'PhotoAmt' 'AdoptionSpeed' 'dataset_type' 'Tiene_nombre?'
 'Tiene_descripcion?' 'MaturitySize_1' 'MaturitySize_2' 'MaturitySize_3'
 'MaturitySize_4' 'Vaccinated_1' 'Vaccinated_2' 'Vaccinated_3'
 'Dewormed_1' 'Dewormed_2' 'Dewormed_3' 'Sterilized_1' 'Sterilized_2'
 'Sterilized_3' 'Health_1' 'Health_2' 'Health_3' 'Vaccinated&Dewormed'
 'Vaccinated&Sterilized' 'Sterilized&Dewormed'
 'Sterilized&Dewormed&Vaccinated' 'Hasphoto?' 'Hasvideo?'
 'Hasvideo&photo?']


## 5. Normalizar o Estandarizar las variables Numericas (para los modelos que sean necesarios)

Revisar si existen valores extremos y considerarlos para los modelos que afecte

In [15]:
#para este modelo no vamos a necesitar esto.

## 6. Separa la base de Test (10%) y Train (90%)
Pueden ser otros porcentajes que les parezcan mejor

In [16]:
train, test = model_selection.train_test_split(data, test_size=0.1, random_state=0)
train.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,...,Dewormed_2,Dewormed_3,Sterilized_1,Sterilized_2,Sterilized_3,Health_1,Health_2,Health_3,Vaccinated&Dewormed,Vaccinated&Sterilized,Sterilized&Dewormed,Sterilized&Dewormed&Vaccinated,Hasphoto?,Hasvideo?,Hasvideo&photo?
6958,1,1,307,0,3,2,5,7,1,1,2,2,2,1,5,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0,0,0,1,0,0
7302,1,51,205,0,2,2,7,0,2,1,1,1,1,1,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1,1,1,1,0,0
12736,1,43,307,0,3,2,0,0,2,1,1,1,1,1,2,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,1,1,1,1,0,0
13287,1,4,307,0,1,1,0,0,3,2,1,1,2,1,1,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,0,0,0,1,0,0
5105,2,4,266,0,3,1,6,7,1,1,2,2,2,1,3,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0,0,0,1,0,0


### 7. Para la parte de Train, armar un esquema de Cross Validation

Usar 10 Folds

In [17]:
kf = model_selection.KFold(n_splits=10)
folds = [(train.iloc[train_idx].index, train.iloc[test_idx].index) for train_idx, test_idx in kf.split(train)]
folds[0]

(Index([ 8744,  4448,  2017, 13165,  3889, 13740,   724,  9544,  6693,  1259,
        ...
         4373,  7891,  9225, 14019,  4859, 13123,  3264,  9845, 10799,  2732],
       dtype='int64', length=12143),
 Index([ 6958,  7302, 12736, 13287,  5105,  1082,  1638,  6203,  5931, 13134,
        ...
         5567, 13096,  1083, 13548, 12346,  2457,  9172,  1559,  8003, 12805],
       dtype='int64', length=1350))

## 8. Entrenar al menos un Modelo que prefieran y optimizar al menos un Hiperparámetro

In [20]:
res.loc[str(depth), "fold_" + str(i)] = roc_auc_score(y_valid.loc[probs.index], probs)
res

NameError: name 'probs' is not defined

In [19]:
depths = list(range(2, 40, 2)) + [None] #list(range(2, 40, 2))
res = pd.DataFrame([], index=[str(d) for d in depths], columns=["fold_" + str(i) for i in range(len(folds))] + ["valid"])

for depth in depths:
    valid_probs = []
    test_probs = []
    for i, (train_idx, valid_idx) in enumerate(folds):
        X_train = train.loc[train_idx]
        y_train = X_train.AdoptionSpeed
        X_train = X_train.drop("AdoptionSpeed", axis=1)

        X_valid = train.loc[valid_idx]
        y_valid = X_valid.AdoptionSpeed
        X_valid = X_valid.drop("AdoptionSpeed", axis=1)

        learner = XGBClassifier(n_estimators=10000, max_depth=depth)
        learner.fit(X_train, y_train,  early_stopping_rounds=10, eval_metric="auc",
                    eval_set=[(X_train, y_train),
                              (test.drop("AdoptionSpeed", axis=1), test.AdoptionSpeed),
                              (X_valid, y_valid)])
        probs = pd.Series(learner.predict_proba(X_valid, ntree_limit=learner.best_ntree_limit)[:, -1], index=X_valid.index)
        valid_probs.append(probs)
        res.loc[str(depth), "fold_" + str(i)] = roc_auc_score(y_valid.loc[probs.index], probs, multi_class = 'ovr' )
        probs = pd.Series(learner.predict_proba(test.drop("AdoptionSpeed", axis=1), ntree_limit=learner.best_ntree_limit)[:, -1], index=test.index, name="fold_" + str(i))
        test_probs.append(probs)
    valid_probs = pd.concat(valid_probs)
    res.loc[str(depth), "valid"] = roc_auc_score(train.loc[valid_probs.index, "AdoptionSpeed"], valid_probs, multi_class = 'ovr' )
    test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
    res.loc[str(depth), "test"] = roc_auc_score(test.loc[test_probs.index, "AdoptionSpeed"], test_probs, multi_class = 'ovr' )

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'